In [ ]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

In [ ]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

# Carregando a LLM

In [ ]:
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=500,
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=pipe)

# Template e Chain

In [ ]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respndendo pergunts gerais.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{pergunta}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
prompt = PromptTemplate.from_template(template)
prompt

In [ ]:
chain = prompt | llm
chain.invoke({'pergunta': 'Que dia é hoje?'})

# Prompt para RAG

prompt base: https://smith.langchain.com/hub/rlm/rag-prompt

In [ ]:
template_rag = '''
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Você é um assistente virtual prestativo e está respondendo perguntas gerais.
Use os seguintes pedaços de contexto recuperado para responder à pergunta.
Se você não sabe a resposta, apenas diga que não sabe. Mantenha a respsta concisa.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Pergunta: {pergunta}
Contexto: {contexto}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
'''

In [ ]:
prompt_rag = PromptTemplate.from_template(template_rag)
prompt_rag

## Definindo contexto

In [ ]:
from datetime import date

dia = date.today()
dia

In [ ]:
contexto = f'Você sabe que hoje é dia {dia}'
print(contexto)

## Criação da Chain / Geração

In [ ]:
# Em casos de alucinação: Responda a pergunta com base apenas no contexto
chain_rag = prompt_rag | llm | StrOutputParser()
pergunta = 'Que dia é hoje? Retorne a data em formato dd/mm/yyyy'

chain_rag.invoke({'pergunta': pergunta, 'contexto': contexto})

In [ ]:
chain_rag

In [ ]:
chain_rag = prompt_rag | llm | StrOutputParser()

contexto = '''
Faturamento trimestral:
1º: R$42476,40
2º: R$46212,97
3º: R$41324,56
4º: R$56430,24
'''
pergunta = 'Qual trimestre teve o maior faturamento?'

chain_rag.invoke({'pergunta': pergunta, 'contexto': contexto})

## Depuração / Debugging



In [ ]:
from langchain.globals import set_debug
set_debug(True)

In [ ]:
pergunta = 'Qual trimestre teve o menor faturamento?'

chain_rag.invoke({'pergunta': pergunta, 'contexto': contexto})

In [ ]:
set_debug(False)

# Aplicação para RAG com contextos maiores

## Etapas de Indexação

### 1 - Carregar o conteúdo

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [ ]:
loader = WebBaseLoader(web_paths=('https://www.bbc.com/portuguese/articles/cd19vexw0y1o',))
docs = loader.load()

In [ ]:
len(docs[0].page_content)

In [ ]:
print(docs[0].page_content[:300])

### 2 - Divisão em pedaços de texto / Split

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

In [ ]:
splits[0]

In [ ]:
splits[1]

### 3 - Armazenamento

In [ ]:
hf_embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-mpnet-base-v2')

In [ ]:
input_test = 'Um teste apenas'
result = hf_embeddings.embed_query(input_test)

In [ ]:
len(result)

In [ ]:
print(result)

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)  # armazenamento no formato de vetor

## Etapas de Recuperação e geração de texto

### 4 - Configurand o recuperador de texto / Retriever

In [ ]:
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 6})

### 5 - Geração

In [ ]:
template_rag

In [ ]:
prompt_rag = PromptTemplate(
    input_variables=['pergunta', 'contexto'],
    template=template_rag
)
prompt_rag

In [ ]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [ ]:
chain_rag = ({'contexto': retriever | format_docs, 'pergunta': RunnablePassthrough()}
             | prompt_rag
             | llm
             | StrOutputParser())

In [ ]:
# teste sem RAG
chain.invoke('Qual filme ganhou mais orcars na premiação de 2024?')

In [ ]:
# teste com RAG
chain_rag.invoke('Qual filme ganhou mais orcars na premiação de 2024?')

In [ ]:
chain_rag.invoke('Quem ganhou o prêmio de melhor ator?')

In [ ]:
vectorstore.delete_collection() # todo o contexto será apagado (limpa o banco de dados)